In [ ]:
!pip install streamlit pyngrok flask-cors gdown -q

# 🔑 Paste your ngrok authtoken here (get one free: https://dashboard.ngrok.com/get-started/your-authtoken)
NGROK_AUTH_TOKEN = "32pdo47nOUrEhMkM3qkBKiH3cR1_5HAoYmPj6zfA5t4Km1JKG"

import gdown, os
from pyngrok import ngrok, conf

# Apply ngrok auth
conf.get_default().auth_token = NGROK_AUTH_TOKEN

# Make artifacts folder
os.makedirs("artifacts", exist_ok=True)

# Download model + columns.json from Google Drive
model_file_id = "15mXWQM1vdYQoCM1oUCT2tQloA0T1Uayc"       # random_forest_base.pkl
columns_file_id = "1byDkPrAWp_6bFRheS9o-uMyKdK0JtXuQ"     # columns.json

gdown.download(f"https://drive.google.com/uc?id={model_file_id}", "artifacts/random_forest_base.pkl", quiet=False)
gdown.download(f"https://drive.google.com/uc?id={columns_file_id}", "artifacts/columns.json", quiet=False)

# util.py
util_code = """
import pickle
import json
import numpy as np

__locations = None
__data_columns = None
__model = None

def get_estimated_price(location, bath, balcony, bhk, total_sqft_cleaned):
    try:
        loc_index = __data_columns.index(location.lower())
    except:
        loc_index = -1

    room_size_avg = total_sqft_cleaned / bhk if bhk > 0 else 0
    price_per_sqft = total_sqft_cleaned / bhk if bhk > 0 else 0

    x = np.zeros(len(__data_columns))
    x[0] = bath
    x[1] = balcony
    x[2] = bhk
    x[3] = total_sqft_cleaned
    x[4] = price_per_sqft
    x[5] = room_size_avg

    if loc_index >= 0:
        x[loc_index] = 1

    return round(__model.predict([x])[0], 2)

def load_saved_artifacts():
    global __data_columns, __locations, __model
    with open("./artifacts/columns.json", "r") as f:
        __data_columns = json.load(f)["data_columns"]
        __locations = __data_columns[6:]

    if __model is None:
        with open("./artifacts/random_forest_base.pkl", "rb") as f:
            __model = pickle.load(f)

def get_location_names():
    return __locations

def get_data_columns():
    return __data_columns
"""
with open("util.py", "w") as f:
    f.write(util_code)

# app.py
app_code = """
import streamlit as st
import util

util.load_saved_artifacts()
st.set_page_config(page_title="House Price Prediction", layout="centered")

st.title("🏠 House Price Prediction App")

location = st.selectbox("Select Location", util.get_location_names())
bath = st.number_input("Number of Bathrooms (bath)", min_value=1, step=1)
balcony = st.number_input("Number of Balconies", min_value=0, step=1)
bhk = st.number_input("BHK (Bedrooms)", min_value=1, step=1)
total_sqft = st.number_input("Total Sqft Cleaned", min_value=100.0, step=10.0)

if st.button("Predict Price"):
    if bhk > 0 and total_sqft > 0:
        price = util.get_estimated_price(location, bath, balcony, bhk, total_sqft)
        st.success(f"💰 Estimated Price: ₹ {price:,.2f}")
    else:
        st.error("❌ Invalid input: Sqft and BHK must be greater than 0.")
"""
with open("app.py", "w") as f:
    f.write(app_code)

# Expose Streamlit app
public_url = ngrok.connect(8501)
print("🌍 Public URL (share with recruiters):", public_url)

!streamlit run app.py --server.port 8501 --server.headless true


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 37.7 MB/s eta 0:00:00


Downloading...
From: https://drive.google.com/uc?id=15mXWQM1vdYQoCM1oUCT2tQloA0T1Uayc
To: /content/artifacts/random_forest_base.pkl
100%|██████████| 69.4M/69.4M [00:00<00:00, 154MB/s]
Downloading...
From: https://drive.google.com/uc?id=1byDkPrAWp_6bFRheS9o-uMyKdK0JtXuQ
To: /content/artifacts/columns.json
100%|██████████| 4.07k/4.07k [00:00<00:00, 12.6MB/s]

🌍 Public URL (share with recruiters): NgrokTunnel: "https://6c20d8e2c9d4.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.148.76.43:8501

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestRegressor was fitted with feature names
  warnings.warn(
